In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

In [10]:
# Get the example portfolios from Finbourne-Examples
all_portfolios = atlas.lusid_portfolio()

example_pfs = all_portfolios.select(
    '*'
).where(
    all_portfolios.portfolio_scope == 'Finbourne-Examples'
).to_table_var()

# Tutorial 5 - Joins in Fluent Syntax

## Introduction

All table classes in lumipy have a collection of join methods such as `inner_join` and `left_join`. These can take any source table class except for another join.

The join expressions are flexible enough to accept columns/expressions from any of their parent tables and automatically handle prefixing and aliasing in the case of clashing column names. Table aliases can be specified in the join method (`right_alias`/`left_alias` args) or by supplying aliased tables (see cell below).

In [11]:
portfolios = example_pfs.with_alias('ptf')
holdings = atlas.lusid_portfolio_holding().with_alias('hld')
instruments = atlas.lusid_instrument().with_alias('ins')
properties = atlas.lusid_property().with_alias('prp')

## Simple Join
In this example we'll join example portfolios and holdings on portfolio code with the additional condition that holdings are in the scopre Finbourne-Examples (this speeds the query up).

Note that the `on` argument will take any expression that is made out of columns from the parent tables and resolves to a boolean. 

In [12]:
join = portfolios.left_join(
    holdings,
    on=(holdings.portfolio_code == portfolios.portfolio_code) &
       (holdings.portfolio_scope == 'Finbourne-Examples')
)

qry = join.select('^')

In [13]:
df = qry.go()
df.head()

,BaseCurrency,PortfolioCode_ptf,PortfolioScope_ptf,PortfolioType,CostAmount,CostCurrency,Error,HoldingType,LusidInstrumentId,PortfolioCode_hld,PortfolioScope_hld,SettledUnits,Units
0,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,-67524816.67,USD,NaN,Position,LUID_O05318VH,US-Corporate-Bond,Finbourne-Examples,-6.500000e+07,-6.500000e+07
1,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,-41063660.78,USD,NaN,Position,LUID_JDC8O2UN,US-Corporate-Bond,Finbourne-Examples,-4.014706e+07,-4.014706e+07
2,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,-31362013.46,USD,NaN,Position,LUID_EOLLTT5M,US-Corporate-Bond,Finbourne-Examples,-2.838971e+07,-2.838971e+07
3,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,-5625523.49,USD,NaN,Position,LUID_755URDIA,US-Corporate-Bond,Finbourne-Examples,-5.520221e+06,-5.520221e+06
4,USD,US-Corporate-Bond,Finbourne-Examples,Transaction,-764876.15,USD,NaN,Balance,CCY_USD,US-Corporate-Bond,Finbourne-Examples,-7.648761e+05,-7.648761e+05


## Chaining Joins Together

You can build up more complex joins with more than two tables by chaining join methods together. In the example below we join on instrument properties to the portfolio holdings.

In [14]:
join = portfolios.left_join(
    holdings,
    on=(portfolios.portfolio_code == holdings.portfolio_code) &
       (holdings.portfolio_scope == 'Finbourne-Examples')
).left_join(
    properties,
    on=(properties.domain == 'Instrument') &
       (properties.entity_id_type == 'LusidInstrumentId') &
       (properties.entity_id == holdings.lusid_instrument_id)
)

## Filtering and Aggregating Joins

Once you have a join table built you can then call select() and chain the other query methods as usual. The example below groups by instrument ID, counts the number of properties and sorts by that count. 

In [15]:
qry = join.select(
    portfolios.portfolio_code,
    holdings.lusid_instrument_id
).group_by(
    holdings.lusid_instrument_id
).aggregate(
    NumProperties=properties.entity_id.count() 
).order_by(
    properties.entity_id.count().descending()
)

df = qry.go()
df.head()

,PortfolioCode,LusidInstrumentId,NumProperties
0,UK-Equities,LUID_YU1CI1C0,2
1,UK-Equities,LUID_4CINGMZM,2
2,US-Treasury-Bond,LUID_Y1XGBGYY,1
3,US-Corporate-Bond,LUID_XB11B4VW,1
4,Global-Equity,LUID_WW55WKWV,1
